# Acquire
* How/where did you get your data? **MySQL, read query into pandas DataFrame**
* What is the size of your data? **7043 rows, 24 columns**
* What does each observation represent? **A customer**
* What does each column represent? **(See [Data Dictionary](#data-dictionary))**

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import env
import os

In [3]:
def get_connection(db, user=env.user, host=env.host, password=env.pwd):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_telco_data():
    filename = "data/telco_churn_raw.csv"

    # Read the csv file if it exists
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # Fead the SQL query into a dataframe
        df = pd.read_sql('''SELECT * 
                            FROM customers
                             JOIN contract_types USING (contract_type_id)
                             JOIN internet_service_types USING (internet_service_type_id)
                             JOIN payment_types USING (payment_type_id)''',
                         get_connection('telco_churn'))
        # Write that DataFrame for prep
        df.to_csv(filename, index=False)
        # Return the DataFrame
        return df

In [4]:
telco_raw = get_telco_data()
print(telco_raw.shape)
telco_raw.head(3)

(7043, 24)


,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check


# Prepare

1. Inspect raw data and note any desired transformations which may include any of the following:
    * Drop unnecessary columns (duplicate, redundant columns)
    * Numeric columns should be numeric data types
    * Handle missing values and impute appropriate values
        * check for explicit missing values (e.g. `np.nan`)
        * check for implicit missing values (e.g. whitespace, `'unknown'`, etc.)
    * Deal with any duplicate rows
    * Address and encode categorical columns
        * one-hot encode unordered categorical columns
        * label encode ordered categorical columns  
1. Inspect clean data
    * Ensure data is tidy:
        * one value per cell
        * each observation is one and only one row
        * each feature is one and only one column
1. Split the data
    * Determine if target column has class imbalance. If, so stratify.
1. Create data dictionary
1. Summarize data transformations

In [5]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import acquire as a
from sklearn.model_selection import train_test_split

In [6]:
telco_raw = a.get_telco_data()
telco_raw.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,Yes,No,Yes,Yes,Yes,98.0,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,No,Yes,Yes,No,Yes,83.9,267.4,Yes,Month-to-month,Fiber optic,Mailed check


#### Inspect Raw Data

In [7]:
display(telco_raw.info())
telco_raw.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

None

,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692
std,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


Takeaways and desired transformations:
- Drop foreign key columns (duplicate columns)
- No explicit null-values
- See why the total_charges column is an object, and convert it to a float, if appropriate.
    - May contain implicit missing values or undesired strings.
- Check for duplicate rows
- Address and encode categorical columns

###### Drop foreign key columns

In [8]:
# Drop foreign keys
telco = telco_raw.iloc[:,3:]

###### Handle Missing Values

- No null values, but I will check for empty string inputs.

In [9]:
def identify_cols_with_white_space(df):
    '''
    takes in a DataFrame
    prints columns with any values that are whitespace
    returns columns in a list
    '''
    
    cols_w_white_space = []
    
    for col in df.columns:
        # check string/object columns
        if df[col].dtype == 'O':
            # check for any values in the column that are empty or whitespace
            is_whitespace = df[col].str.isspace()
            has_whitespace = is_whitespace.any()
            if has_whitespace:
                print(f'{col} has whitespace')
                cols_w_white_space.append(col)
    return cols_w_white_space

In [10]:
identify_cols_with_white_space(telco)

total_charges has whitespace


['total_charges']

The total_charges columns has whitespace values, which explains why the column isn't a float type, as expected

In [11]:
telco['total_charges'].value_counts().head(2)

        11
20.2    11
Name: total_charges, dtype: int64

There are 11 whitespace values. To resolve this issue, I can either impute a reasonable value, impute a missing value (`np.nan`), or drop the rows with this value. With thousands of observations and very few values to impute, it is reasonable to impute the median total_charges.

In [12]:
# replace empty space with np.nan and convert column to float
telco['total_charges'] = telco['total_charges'].replace(' ', np.nan).astype(float)
# impute median
telco['total_charges'] = telco['total_charges'].fillna(telco['total_charges'].median())
telco['total_charges'].value_counts(dropna=False).head(2)

1397.475    11
20.200      11
Name: total_charges, dtype: int64

###### Handle Duplicate Rows

In [13]:
# check duplicate rows
telco.duplicated().sum()

0

In [14]:
# check for duplicate customer id's
telco['customer_id'].duplicated().sum()

0

###### Address Catergorical Columns

- encode

In [15]:
# getting value counts of categorical columns
for col in telco.columns:
    if len(telco[col].unique()) < 10:
        display(telco[col].value_counts(dropna=False))


Male      3555
Female    3488
Name: gender, dtype: int64

0    5901
1    1142
Name: senior_citizen, dtype: int64

No     3641
Yes    3402
Name: partner, dtype: int64

No     4933
Yes    2110
Name: dependents, dtype: int64

Yes    6361
No      682
Name: phone_service, dtype: int64

...

Yes    4171
No     2872
Name: paperless_billing, dtype: int64

No     5174
Yes    1869
Name: churn, dtype: int64

Month-to-month    3875
Two year          1695
One year          1473
Name: contract_type, dtype: int64

Fiber optic    3096
DSL            2421
None           1526
Name: internet_service_type, dtype: int64

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: payment_type, dtype: int64

One-hot encode all columns

- manually encode binary class columns
- get dummy variables for multiclass columns

In [16]:
# Encoding binary variables
binary_cols = ['partner','dependents','phone_service', 'paperless_billing', 'churn']
for col in binary_cols:
    telco[col] = telco[col].replace({'Yes': 1, 'No': 0})

telco['gender'] = telco['gender'].replace({'Male': 1, 'Female': 0})

In [17]:
# Encoding multiclass variables
dummy_df = pd.get_dummies(telco[['multiple_lines', 'online_security', 'online_backup',
                                 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                                 'contract_type', 'internet_service_type', 'payment_type']],
                                  dummy_na=False, drop_first=True)
telco_clean = pd.concat([telco, dummy_df], axis=1)

In [18]:
telco_clean.head(3)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,device_protection_No internet service,device_protection_Yes,tech_support_No internet service,tech_support_Yes,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0002-ORFBO,0,0,1,1,9,1,No,No,Yes,No,Yes,Yes,No,1,65.6,593.30,0,One year,DSL,Mailed check,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
1,0003-MKNFE,1,0,0,0,9,1,Yes,No,No,No,No,No,Yes,0,59.9,542.40,0,Month-to-month,DSL,Mailed check,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0004-TLHLJ,1,0,0,0,4,1,No,No,No,Yes,No,No,No,1,73.9,280.85,1,Month-to-month,Fiber optic,Electronic check,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0


Data looks sufficient and tidy. I will rename the columns appropriately and do a final inspection.

In [19]:
# rename columns to be lowercased with underscores
telco_clean.columns = [col.lower().replace(" ", "_") for col in telco_clean.columns]
telco_clean = telco_clean.rename(columns={'gender':'gender_male'})

In [20]:
display(telco_clean.info())
telco_clean.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 42 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customer_id                            7043 non-null   object 
 1   gender_male                            7043 non-null   int64  
 2   senior_citizen                         7043 non-null   int64  
 3   partner                                7043 non-null   int64  
 4   dependents                             7043 non-null   int64  
 5   tenure                                 7043 non-null   int64  
 6   phone_service                          7043 non-null   int64  
 7   multiple_lines                         7043 non-null   object 
 8   online_security                        7043 non-null   object 
 9   online_backup                          7043 non-null   object 
 10  device_protection                      7043 non-null   object 
 11  tech

None

,gender_male,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,multiple_lines_no_phone_service,multiple_lines_yes,online_security_no_internet_service,online_security_yes,online_backup_no_internet_service,online_backup_yes,device_protection_no_internet_service,device_protection_yes,tech_support_no_internet_service,tech_support_yes,streaming_tv_no_internet_service,streaming_tv_yes,streaming_movies_no_internet_service,streaming_movies_yes,contract_type_one_year,contract_type_two_year,internet_service_type_fiber_optic,internet_service_type_none,payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.592219,64.761692,2281.916928,0.265370,0.096834,0.421837,0.216669,0.286668,0.216669,0.344881,0.216669,0.343888,0.216669,0.290217,0.216669,0.384353,0.216669,0.387903,0.209144,0.240664,0.439585,0.216669,0.216101,0.335794,0.228880
std,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.491457,30.090047,2265.270398,0.441561,0.295752,0.493888,0.412004,0.452237,0.412004,0.475363,0.412004,0.475038,0.412004,0.453895,0.412004,0.486477,0.412004,0.487307,0.406726,0.427517,0.496372,0.412004,0.411613,0.472301,0.420141
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.500000,402.225000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.350000,1397.475000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.850000,3786.600000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,8684.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


###### Investigate 204 customers (rows) with tenure of 72

Potential duplicate customers with different IDs

In [21]:
telco_clean[telco_clean['tenure'] == 72]

,customer_id,gender_male,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,multiple_lines_no_phone_service,multiple_lines_yes,online_security_no_internet_service,online_security_yes,online_backup_no_internet_service,online_backup_yes,device_protection_no_internet_service,device_protection_yes,tech_support_no_internet_service,tech_support_yes,streaming_tv_no_internet_service,streaming_tv_yes,streaming_movies_no_internet_service,streaming_movies_yes,contract_type_one_year,contract_type_two_year,internet_service_type_fiber_optic,internet_service_type_none,payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
11,0017-IUDMW,0,0,1,1,72,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,1,116.80,8456.75,0,Two year,Fiber optic,Credit card (automatic),0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,0
13,0019-EFAEP,0,0,0,0,72,1,Yes,Yes,Yes,Yes,No,Yes,No,1,101.30,7261.25,0,Two year,Fiber optic,Bank transfer (automatic),0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0
45,0078-XZMHT,1,0,1,0,72,1,Yes,No,Yes,Yes,Yes,Yes,Yes,1,85.15,6316.20,0,Two year,DSL,Bank transfer (automatic),0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0
79,0129-KPTWJ,1,0,1,0,72,1,No,No,No,Yes,No,Yes,Yes,1,94.65,6747.35,0,Month-to-month,Fiber optic,Electronic check,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0
95,0164-APGRB,0,0,0,0,72,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,1,114.90,8496.70,0,Two year,Fiber optic,Electronic check,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,9880-TDQAC,0,0,1,1,72,0,No phone service,No,Yes,Yes,Yes,Yes,Yes,1,60.00,4264.00,0,Two year,DSL,Electronic check,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0
6987,9919-FZDED,1,1,1,0,72,1,Yes,Yes,Yes,Yes,No,Yes,Yes,1,84.10,5981.65,0,Two year,DSL,Credit card (automatic),0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0
6993,9924-JPRMC,1,0,0,0,72,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,1,118.20,8547.15,0,Two year,Fiber optic,Electronic check,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0
6994,9926-PJHDQ,0,0,1,1,72,1,Yes,No,No,Yes,No,Yes,Yes,1,76.80,5468.45,0,Two year,DSL,Bank transfer (automatic),0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0


Rows are different, so no duplicate customers

Create generalized function:

In [29]:
col

'churn'

In [28]:
telco_raw[col].nunique()

2

In [1]:
def prep_telco_explore(telco_raw=a.get_telco_data(), target='churn'):
    '''
    Accepts the raw telco data
    Returns the data with the transformations above applied
    '''
    
    # drop foreign keys
    telco = telco_raw.iloc[:,3:]

    # replace empty space with np.nan and convert column to float
    telco['total_charges'] = telco['total_charges'].replace(' ', np.nan).astype(float)
    # impute median
    telco['total_charges'] = telco['total_charges'].fillna(telco['total_charges'].median())

    # decode senior column
    telco['senior_citizen'] = telco['senior_citizen'].replace({1: 'Senior', 0: 'Non-Senior'})

    # encode multiclass variables and target variable
    telco[target] = telco[target].replace({'Yes': 1, 'No': 0})
    dummy_df = pd.get_dummies(telco[[col for col in telco.columns
                                     if (telco[col].nunique() > 2 and telco[col].nunique() < 10)]])
    dummy_df = dummy_df.replace({0: 'No', 1: 'Yes'})
    telco_explore = pd.concat([telco, dummy_df], axis=1)

    # rename columns to be lowercased with underscores
    telco_explore.columns = [col.lower().replace(" ", "_") for col in telco_explore.columns]

    # Write the clean DataFrame for exploration and modeling
    filename = "data/telco_churn_explore.csv"
    telco_explore.to_csv(filename, index=False)    

    return telco_explore

def prep_telco_model(telco_raw=a.get_telco_data()):
    '''
    Accepts the raw telco data
    Returns the data with the transformations above applied
    '''
    
    # drop foreign keys
    telco = telco_raw.iloc[:,3:]

    # replace empty space with np.nan and convert column to float
    telco['total_charges'] = telco['total_charges'].replace(' ', np.nan).astype(float)
    # impute median
    telco['total_charges'] = telco['total_charges'].fillna(telco['total_charges'].median())

    # encode binary variables
    binary_cols = ['partner','dependents','phone_service', 'paperless_billing', 'churn']
    for col in binary_cols:
        telco[col] = telco[col].replace({'Yes': 1, 'No': 0})

    telco['gender'] = telco['gender'].replace({'Male': 1, 'Female': 0})

    # encode multiclass columns
    multiclass_cols = [col for col in telco.columns
                       if (telco[col].nunique() > 2 and telco[col].nunique() < 10)]
    dummy_df = pd.get_dummies(telco[multiclass_cols], drop_first=True)
    telco_model = pd.concat([telco, dummy_df], axis=1)
    
    # drop multiclass columns
    telco_model = telco_model.drop(columns=multiclass_cols)

    # rename columns to be lowercased with underscores
    telco_model.columns = [col.lower().replace(" ", "_") for col in telco_model.columns]
    # rename gender column to gender_male
    telco_model = telco_model.rename(columns={'gender':'gender_male'})

    # Write the clean DataFrame for exploration and modeling
    filename = "data/telco_churn_model.csv"
    telco_model.to_csv(filename, index=False)    

    return telco_model

NameError: name 'a' is not defined

In [59]:
telco_explore = prep_telco_explore()

In [60]:
telco_explore.head(3)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,multiple_lines_no,multiple_lines_no_phone_service,multiple_lines_yes,online_security_no,online_security_no_internet_service,online_security_yes,online_backup_no,online_backup_no_internet_service,online_backup_yes,device_protection_no,device_protection_no_internet_service,device_protection_yes,tech_support_no,tech_support_no_internet_service,tech_support_yes,streaming_tv_no,streaming_tv_no_internet_service,streaming_tv_yes,streaming_movies_no,streaming_movies_no_internet_service,streaming_movies_yes,contract_type_month-to-month,contract_type_one_year,contract_type_two_year,internet_service_type_dsl,internet_service_type_fiber_optic,internet_service_type_none,payment_type_bank_transfer_(automatic),payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
0,0002-ORFBO,Female,Non-Senior,Yes,Yes,9,Yes,No,No,Yes,No,Yes,Yes,No,Yes,65.6,593.30,No,One year,DSL,Mailed check,Yes,No,No,Yes,No,No,No,No,Yes,Yes,No,No,No,No,Yes,No,No,Yes,Yes,No,No,No,Yes,No,Yes,No,No,No,No,No,Yes
1,0003-MKNFE,Male,Non-Senior,No,No,9,Yes,Yes,No,No,No,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check,No,No,Yes,Yes,No,No,Yes,No,No,Yes,No,No,Yes,No,No,Yes,No,No,No,No,Yes,Yes,No,No,Yes,No,No,No,No,No,Yes
2,0004-TLHLJ,Male,Non-Senior,No,No,4,Yes,No,No,No,Yes,No,No,No,Yes,73.9,280.85,Yes,Month-to-month,Fiber optic,Electronic check,Yes,No,No,Yes,No,No,Yes,No,No,No,No,Yes,Yes,No,No,Yes,No,No,Yes,No,No,Yes,No,No,No,Yes,No,No,No,Yes,No


In [61]:
telco_model = prep_telco_model()

In [62]:
telco_model.head(3)

,customer_id,gender_male,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,multiple_lines_no_phone_service,multiple_lines_yes,online_security_no_internet_service,online_security_yes,online_backup_no_internet_service,online_backup_yes,device_protection_no_internet_service,device_protection_yes,tech_support_no_internet_service,tech_support_yes,streaming_tv_no_internet_service,streaming_tv_yes,streaming_movies_no_internet_service,streaming_movies_yes,contract_type_one_year,contract_type_two_year,internet_service_type_fiber_optic,internet_service_type_none,payment_type_credit_card_(automatic),payment_type_electronic_check,payment_type_mailed_check
0,0002-ORFBO,0,0,1,1,9,1,1,65.6,593.30,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
1,0003-MKNFE,1,0,0,0,9,1,0,59.9,542.40,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0004-TLHLJ,1,0,0,0,4,1,1,73.9,280.85,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0


###### Split the data

In [80]:
def split_data(df, test_size=.15, validate_size=.15, stratify_col=None, random_state=None):
    '''
    Takes in a DataFrame, desired test and validate sizes, an optional column to stratify on and a random state;
    Returns train, validate, and test DataFrames.
    '''
    # no stratification
    if stratify_col == None:
        # split test data
        train_validate, test = train_test_split(df, test_size=test_size, random_state=random_state)
        # split validate data
        train, validate = train_test_split(train_validate, test_size=validate_size/(1-test_size),
                                                                           random_state=random_state)
    # stratify split
    else:
        # split test data
        train_validate, test = train_test_split(df, test_size=test_size,
                                                random_state=random_state, stratify=df[stratify_col])
        # split validate data
        train, validate = train_test_split(train_validate, test_size=validate_size/(1-test_size),
                                           random_state=random_state, stratify=train_validate[stratify_col])       
    return train, validate, test

Check for class imbalance to see if we should do a stratified split.

In [81]:
telco_prep['churn'].value_counts(normalize=True)

0    0.73463
1    0.26537
Name: churn, dtype: float64

73%:27% non-churn to churn ratio, so we will stratify.

In [86]:
train, validate, test = split_data(telco_prep, stratify_col='churn', random_state=125)

4929 1057 1057


0    0.734632
1    0.265368
Name: churn, dtype: float64

Verify split

In [ ]:
# check set sizes
print(len(train), len(validate), len(test))
# check target proportions match original data (stratified)
train['churn'].value_counts(normalize=True)

###### Create Data Dictionary

<a id="data-dictionary"></a>

Data Dictionary:

| Feature              | Definition                                                   |
|:---------------------|:-------------------------------------------------------------|
| customer_id          | Unique identifier for each customer                           |
| gender_male          | Indicates whether the customer is male or not                 |
| senior_citizen       | Indicates whether the customer is a senior citizen or not     |
| partner              | Indicates whether the customer has a partner or not           |
| dependents           | Indicates whether the customer has dependents or not          |
| tenure               | Number of months the customer has been with the company       |
| phone_service        | Indicates whether the customer has phone service or not       |
| multiple_lines       | Indicates whether the customer has multiple lines for phone   |
| online_security      | Indicates whether the customer has online security service or not |
| online_backup        | Indicates whether the customer has online backup service or not |
| device_protection    | Indicates whether the customer has device protection service or not |
| tech_support         | Indicates whether the customer has tech support service or not |
| streaming_tv         | Indicates whether the customer has streaming TV service or not |
| streaming_movies     | Indicates whether the customer has streaming movies service or not |
| paperless_billing    | Indicates whether the customer has opted for paperless billing or not |
| monthly_charges      | The amount charged to the customer on a monthly basis         |
| total_charges        | The total amount charged to the customer over the entire tenure |
| churn                | Indicates whether the customer has churned (cancelled the service) or not |
| contract_type        | The type of contract the customer has (e.g., month-to-month, one-year, two-year) |
| internet_service_type| The type of internet service the customer has (e.g., DSL, fiber optic, None) |
| payment_type         | The method of payment used by the customer (e.g., electronic check, credit card, bank transfer, mailed check) |


###### Summarize Data Transformations

1. Inspected raw data
    * Dropped foreign key columns
    * Checked for missing values
       * found `' '` in the `total_charges` column and replaced them with `np.nan`s
       * converted ` total_charges` column to a numeric data type and imputed `np.nan`s with the median total charges
    * Checked for duplicate rows and customer id’s and saw none
    * Encoded categorical columns

1. Inspect clean data
    * Ensured data was tidy:
        * one value per cell
        * each observation is one and only one row
        * each feature is one and only one column

1. Split the data
    * Saw class imbalance (73% to 27%) for the target, `churn`.
    * Performed 70/15/15 (train/validate/test) stratified split with a random state of 125